## Process species names that produced errors in bacdive API search by using genus name only
E.g. Synechococcus sp.

In [ ]:
import bacdive
import pandas as pd
import numpy as np

In [ ]:
with open('up.txt', 'r') as file:
    creds = [line.rstrip() for line in file]

In [ ]:
client = bacdive.BacdiveClient(creds[0], creds[1])

In [ ]:
##read in error file and remove sp. and numbers
fileerrors = 'species_not_processed.txt'
with open(fileerrors, 'r') as file:
    errors = [line.rstrip() for line in file]
len(errors)

In [ ]:
genus_names = []
for item in errors:
    if 'sp.' in item:
        item = item.replace('sp.', '')
        item = item.strip()
        genus_names.append(item)
len(genus_names)

In [ ]:
##test with genus name
filter = ['NCBI tax id', 'BacDive-ID', 'DSM-Number', 'species', 'Physiology and metabolism', 'Isolation, sampling and environmental information']
try:
    query = {'taxonomy': 'Synechococcus'}
    client.search(**query)
    result = client.retrieve(filter)
except Exception as e:
    print(e)
for item in result:
    print(item)

In [ ]:
##Search the filtered data for each unique species name in Bacdive and return NCBI ID, physiology & metabolism, and isolation data
filter = ['NCBI tax id', 'BacDive-ID', 'DSM-Number', 'species', 'Physiology and metabolism', 'Isolation, sampling and environmental information', 'Culture and growth conditions']
bacdive_data = {}
count = 0
error = []
for genus in genus_names:
    bacdive_data[genus] = []
    try:
        query = {'taxonomy': genus}
        client.search(**query)
        result = client.retrieve(filter)
        for item in result:
            bacdive_data[genus].append(item)
        #time.sleep(3)
    except Exception as e:
        error.append((e, genus))
    count += 1
    if count%100 == 0:
        print(count)
len(bacdive_data)

In [ ]:
##save data dictionary with new results
import pickle
with open('bacdivedata_genus_20230505.pickle', 'wb') as filep:
    pickle.dump(bacdive_data, filep)

In [ ]:
len(error)